In [12]:
import sys
import os
import pickle as pkl
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

sys.path.append("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/Splycer")
from record_set import RecordDataFrame
from pairs_set import PairsCSR
from feature_engineer import FeatureEngineer
from xgboost_match import XGBoostMatch
os.chdir("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/example")

In [13]:
#Load in the census data.
delaware_1910 = RecordDataFrame(2, pd.read_csv("delaware_1910.csv", index_col="index"))
delaware_1920 = RecordDataFrame(3, pd.read_csv("delaware_1920.csv", index_col="index"))

In [14]:
#Load in the compares set
compares = pd.read_csv("delaware_compares.csv")
compares = PairsCSR([1,2], compares.index_1910.values, compares.index_1920.values, np.ones(compares.shape[0]))

In [7]:
#Construct the feature engineer
years = [1,2]
feature_engineer = FeatureEngineer()
features = ["imputer", "bool match", "euclidean distance", "jaro-winkler", 
            "ngram", "drop", "commonality weight"] #FIXME add immigration imputer to FeatureEngineer
first_vec = [f"first_vec{i}" for i in range(2, 202)]
last_vec = [f"last_vec{i}" for i in range(2,202)]
cols_to_drop = ["marstat", "birth_year", "household", "immigration", "race", "rel", "female",
                "bp", "mbp", "fbp", "state", "county", "cohort1", 
                "cohort2", "last_sdxn", "first_sdxn", "last_init",
                "first_init", "bp_comm", "first_comm", "last_comm", "bp_lat", "bp_lon",
                "res_lat", "res_lon", "full_name"]
cols_to_drop.extend(first_vec)
cols_to_drop.extend(last_vec)
feature_params = [{"cols": ["first_comm", "last_comm"], "years": years}, 
                  {"vars_to_match": ["marstat", "race", "rel", "mbp", "fbp", "first_sdxn", "last_sdxn", "bp", "county"], "years": years},
                  {"variables": [["immigration"], ["birth_year"], ["res_lat", "res_lon"], ["bp_lat", "bp_lon"], [f"first_vec{i}" for i in range(2, 202)], [f"last_vec{i}" for i in range(2,202)]], "new_cols": ["immigration_dist", "birth_year_dist", "geodist", "bp_geodist", "first_vec_dist", "last_vec_dist"], "years": years},
                  {"string_col": ["first", "last"], "dist_metric": "jw", "years": years},
                  {"string_col": ["first", "last"], "n": 2, "years": years},
                  {"cols_to_drop": ["first", "last"], "both_years": True, "years": years},
                  {"cols": ["first_jw", "last_jw", "first_sdxn_match", "last_sdxn_match", "bp_match"], "comm_cols": ["first", "last", "first", "last", "bp"], "years": years}]

for i, j in zip(features, feature_params):
    feature_engineer.add_feature(i, j) #this adds features to your feature object. when you run feature_engineer.create_features(), it computes all of the features above.
feature_engineer.build()

In [ ]:
#Construct the feature engineer
fe = FeatureEngineer()
#FIXME add parent geo-coordinates
exact_match_cols = [("marstat",), ("race",), ("rel",), ("mbp",), ("fbp",), ("county",)]
euclid_cols = [("immigration",), ("birth_year",), tuple(f"first_vec{i}" for i in range(2, 202)), 
               tuple(f"last_vec{i}" for i in range(2, 202))]
geodist_cols = [("res_lat", "res_lon"), ("bp_lat", "bp_lon")]
comm_weight_cols = [("bp",), ("first",), ("last",), ("first",), ("last",), ("first_sdxn",), ("last_sdxn",)]
comm_weight_comps = [("exact match", "commonality weight"), ("jw", "commonality weight"), ("jw", "commonality weight"),
                     ("bigram", "commonality weight"), ("bigram", "commonality weight"),
                     ("exact match", "commonality weight"), ("exact match", "commonality weight")]
comm_weight_extra_args = [({}, {"comm_col": "bp_comm"}), ({}, {"comm_col": "first_comm"}), ({}, {"comm_col": "last_comm"}),
                          ]

In [8]:
#Construct XGBoostMatch
with open("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/model.xgboost", "rb") as file:
    model = pkl.load(file)
xgb = XGBoostMatch(delaware_1910, delaware_1920, compares, feature_engineer, model)

In [9]:
xgb.run("test.csv")

ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields full_name_1, pr_name_gn_1, pr_name_surn_1, full_name_2, pr_name_gn_2, pr_name_surn_2

In [4]:
delaware_1910.df.head()

In [6]:
delaware_1910.get_record(8361303)

,marstat,birth_year,household,immigration,race,rel,female,bp,mbp,fbp,...,last_vec192,last_vec193,last_vec194,last_vec195,last_vec196,last_vec197,last_vec198,last_vec199,last_vec200,last_vec201
index,,,,,,,,,,,,,,,,,,,,,
8361303,1.0,1874.0,1934610.0,NaN,0,1,1,65,46,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
